In [8]:
import pandas as pd
import openai
import csv
import time
import os
import regex as re
import math
import random

import openai

pd.set_option('display.max_colwidth', None)
file_path = "D:/api_key.txt"
with open(file_path, 'r') as file:
    api_key = file.read()

openai.api_key = (api_key)

In [9]:
def genereer_data(keywords, voorbeeld_zin, voorbeeld_zin2, temp =1, top_p=0.9, n=5, pp=0):
    generate_correct = '''Genereer medische tekst. De tekst is kort en bondig geschreven en 
    brengt de essentie over van relevante gebeurtenissen van de dag of dagdeel. 
    Het gaat over een oudere client die zorg ontvangt. De tekst is geschreven door een 
    zorgprofessional die geen moeilijke woorden gebruikt. 
    Bijvoorbeeld: ''' + voorbeeld_zin + "Bijvoorbeeld: " + voorbeeld_zin2
    message_history = []
    message_history.append({'role' : "system", "content": generate_correct})
    message_history.append({"role": "user", 'content' : 'genereer 1 tekst met de volgende woorden: '+ keywords})

    completion = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',messages=message_history,
        temperature=temp,
        top_p=top_p,
        presence_penalty=0,
        n=n
    )
    return completion

In [10]:
def sample_from_df(df):
    random_row = df.sample()
    # extract the values of the sampled row
    values = random_row.values[0]
    # print the values as a string
    voorbeeld_zin = (''.join(map(str, values)))

    same = True
    while same == True:
        random_row = df.sample()
        # extract the values of the sampled row
        values = random_row.values[0]
        # print the values as a string
        voorbeeld_zin2= (''.join(map(str, values)))
        if voorbeeld_zin != voorbeeld_zin2:
            same = False

    return voorbeeld_zin, voorbeeld_zin2

In [11]:
def sample_from_examples_length_based(df, length_column_name, text_column_name, length, length_range = 2):
    filtered_df = df[(df[length_column_name] >= (length -length_range)) & (df[length_column_name] <= (length +length_range))]

    example_1 = filtered_df[text_column_name].sample().values[0]

    same = True
    while same == True:
        example_2 = filtered_df[text_column_name].sample().values[0]

        if example_2 != example_1:
            same = False

    return example_1, example_2


In [12]:
def generate_keyword_data(df, output_file_path, keywords_df, amount_per_combination, temp, top_p, generation_limit, pp):
    # check if output file already exists
    if os.path.exists(output_file_path):
        # if it does, read the last row that was successfully processed
        with open(output_file_path, 'r', encoding='utf-8') as output_file:
            reader = csv.reader(output_file)
            for row in reader:
                pass
            last_row_processed = reader.line_num
    else:
        # if it doesn't, set last_row_processed to 0
        last_row_processed = 0

    # open input and output files
    #ISO-8859-1
    with open(output_file_path, 'a', newline='', encoding='utf-8') as output_file:
        writer = csv.writer(output_file)

        #If initializing a new document, skip the header and rewrite the new format.
        if last_row_processed == 0:
            writer.writerow(['Generated', 'Example_1', 'Example_2', 'Temperature', "Top_p", "Pres_penalty", 'Keywords'])
        number_generated = 0
        while generation_limit > number_generated:
            voorbeeld_zin, voorbeeld_zin2 = sample_from_df(df)
            keywords = keywords_df.sample(1).values[0]
            #print(reader.line_num)
            retries = 0
            while retries <= 10:
                if retries == 5:
                    print('failed 5 times in a row.. waiting 30 seconds')
                    time.sleep(30)
                try:
                    completion = genereer_data(voorbeeld_zin, voorbeeld_zin2, n = amount_per_combination, temp=temp, top_p=top_p )
                    #print(reply_content)
                    break
                except:
                    # if API call fails, wait for 10 seconds and retry
                    retries += 1
                    if retries == 10:
                        exit("10 retries failed in a row.")
                    print('failed ', retries, ' times... retrying in 10 seconds')
                    time.sleep(10)
                    #Longer prompt costs more.
                    completion = genereer_data(keywords, voorbeeld_zin, voorbeeld_zin2, n = amount_per_combination, temp=temp, top_p=top_p )
                    break
            for i in range(len(completion.choices)):
                writer.writerow([completion.choices[i].message.content, voorbeeld_zin, voorbeeld_zin2, temp, top_p, pp, keywords])
                number_generated += 1
                print('Rows Processed:', number_generated)

            # flush output file buffer to ensure data is saved
                output_file.flush()

                # print progress
                #print(f"Processed row {last_row_processed}")
                # break out of loop if num_rows is reached
                if generation_limit is not None and number_generated >= generation_limit:
                    print('Generation limit reached')
                    break


In [13]:
df = pd.read_csv("hand_made_correct_medical.csv", sep=',', encoding='ISO-8859-1', header=None)
keyword_df = pd.read_csv('keywords3_v1.csv')
#generated_keywords are saved in v1 to v4 but are all made the same way
generate_keyword_data(df=df, output_file_path='E:/Data_exploration/GitHub/paraphraser_code/data/generated/generated_keyword3_v1.csv',keywords_df=keyword_df['keywords'], amount_per_combination=5, temp=1, top_p=0.9, generation_limit=3200, pp=0)

failed  1  times... retrying in 10 seconds
